In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import cv2
import time
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import Model

from keras_preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import img_to_array

from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import MultiLabelBinarizer, LabelBinarizer
from sklearn.model_selection import train_test_split

import tensorflow_addons as tfa

In [ ]:
df = pd.read_csv('../input/plant-pathology-2021-fgvc8/train.csv')
df.head()

In [ ]:
train = pd.DataFrame(df,columns = ['image','labels'])
train.columns, len(train)

In [ ]:
train['labels'] = train['labels'].apply(lambda s: s.split(' '))
train[:10]

In [ ]:
datagen = ImageDataGenerator(
    rescale = 1./255,
    validation_split= 0.2
)
bsize  = 32

In [ ]:
train_data = datagen.flow_from_dataframe(
    train,
    directory = '../input/resized-plant2021/img_sz_512',
    x_col = 'image',
    y_col = 'labels',
    subset="training",
    color_mode="rgb",
    target_size = (224,224),
    class_mode="categorical",
    batch_size=bsize,
    shuffle=False,
    seed=40,
)

In [ ]:
valid_data = datagen.flow_from_dataframe(
    train,
    directory = '../input/resized-plant2021/img_sz_512',
    x_col = 'image',
    y_col = 'labels',
    subset="validation",
    color_mode="rgb",
    target_size = (224,224),
    class_mode="categorical",
    batch_size=bsize,
    shuffle=False,
    seed=40,
)

In [ ]:
weights_path = '../input/weight/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'
VGG16_MODEL = tf.keras.applications.VGG16(weights=weights_path ,include_top=False, input_shape=(224, 224, 3))

In [ ]:
x=VGG16_MODEL.output
x=GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
prediction=Dense(6, activation='sigmoid')(x)

In [ ]:
model=Model(inputs=VGG16_MODEL.input, outputs=prediction)

In [ ]:
f1_score = tfa.metrics.F1Score(num_classes=6, threshold=0.4, average='micro')
model.compile(tf.keras.optimizers.Adam(learning_rate=0.0005) , loss='binary_crossentropy', metrics=[f1_score])

In [ ]:
start_time=time.time()

model.fit(train_data, epochs=30, validation_data=valid_data)

end_time=time.time()

print('Time taken is ', (end_time-start_time)/60., ' mins'  )

In [ ]:
model.save('vgg_model.h5')